Import modules

In [6]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances


Load the dataset and data management

In [163]:
df = pd.read_excel('df_zouk.xlsx')

# Group by playlist name
groups = df.groupby('playlist_name')


Euclidean distances

In [164]:
scaler = MinMaxScaler()

scaled_columns_dict = {}

for group_name, group in groups:
    # Subset data
    subset = group.loc[:, "danceability":"duration_ms"]

    # Scale the specified columns
    scaled_columns = scaler.fit_transform(subset)

    # Store the scaled columns in dict
    scaled_columns_dict[group_name] = scaled_columns


In [165]:
# Create an empty dictionary to store the distances
results_df = pd.DataFrame()

# Loop over each group
for group_name, group in groups:
    # Calculate the pairwise euclidean distances between the features of each pair of tracks
    features = scaled_columns_dict[group_name]

    pairwise_dist = pairwise_distances(features)

    # Calculate mean distances
    mean_distance = pairwise_dist.mean()
    
    # append the results to the new dataframe
    playlist_followers = group['playlist_followers'].iloc[0]
    track_popularity = group['popularity_track'].mean()
    artist_popularity = group['popularity_artist'].mean()
    nb_tracks = len(group)

    # append the results to the new dataframe
    results_df = results_df.append({'playlist_name': group_name, 'playlist_followers': playlist_followers,
                                    'track_pop': track_popularity, 'artist_pop': artist_popularity,
                                    'nb_tracks': nb_tracks, 'mean_distance': mean_distance
                                    },
                                   ignore_index=True)

# Print the df
print(results_df)


         playlist_name  playlist_followers  track_pop  artist_pop  nb_tracks  \
0        Carnival 2023              203148  37.685714   38.571429         70   
1   Dancehall Official             1013896  48.539326   49.067416         89   
2      Dancehall Party              397519  56.130000   53.990000        100   
3      Dancehall Queen              294345  55.293333   54.866667         75   
4  Dancehall Throwback              521893  58.387500   56.100000         80   
5       Zouk All Stars               44763  43.360000   35.460000         50   
6         Zouk Station               42248  42.400000   36.900000         40   

   mean_distance  
0       0.842901  
1       0.845908  
2       0.881999  
3       0.898493  
4       0.911153  
5       1.008812  
6       1.008712  


/var/folders/41/xvr_rn1j51vd_js7xks_mznh0000gn/T/ipykernel_11688/1995285091.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'playlist_name': group_name, 'playlist_followers': playlist_followers,
/var/folders/41/xvr_rn1j51vd_js7xks_mznh0000gn/T/ipykernel_11688/1995285091.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'playlist_name': group_name, 'playlist_followers': playlist_followers,
/var/folders/41/xvr_rn1j51vd_js7xks_mznh0000gn/T/ipykernel_11688/1995285091.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'playlist_name': group_name, 'playlist_followers': playlist_followers,
/var/folders/41/xvr_rn1j51vd_js7xks_m

Parse data into excel

In [166]:
results_df.to_excel("df_zouk_final_2.xlsx", index=False)
